In [13]:
#!pip install langchain-chroma

In [14]:
#pip install hubspot-api-client

In [22]:
# MAKE A LIST OF THE TOP

In [8]:
pip install pandas



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
hubspot_api_key = os.getenv('HUBSPOT_API_KEY')
google_place_api_key = os.environ["GOOGLE_PLACE_API_KEY"]


print(os.getcwd())
dotenv_path = find_dotenv()
print(f"Loading .env file from: {dotenv_path}")
_ = load_dotenv(dotenv_path)


from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI


from langchain_chroma import Chroma


creativeLlmModel = OpenAI(temperature=0.9)
chatModel = ChatOpenAI(model="gpt-4o-mini")
llmModel = OpenAI()



/Users/kevincorstorphine/Desktop/Agentic_Start
Loading .env file from: /Users/kevincorstorphine/Desktop/Agentic_Start/.env


In [11]:


# GOOGLE PLACES API to find names of businesses in a certain area. Get API Keys and have Google do it.
# Linkedin Sales Navigator to find the contact information of the people (maybe sales nav) ~$99/mo
# Zoominfo to get their people, and then contact information.

# Upload all data to Hubspot CRM

In [12]:
# Accountant
# Law
# Construction
# Manufacturing
# Property management
# Dentists
# Distribution
# Industrial
# Dental
# Optometrist
# Health
# Local_government_office
# Accounting
# Finance
# Veterinary_care
# School
# Museum
# Car_rental
# Insurance_agency


In [16]:
import requests
import pandas as pd
import time


#SEE WHAT ALL DATA YOU CAN INCLUDE IN THE DATABASE
#FIGURE OUT HOW TO BETTER DISTILL THE DATA FOR PCIO. MAYBE JUST TARGET TOP PROSPECTS IN x States?
#GET ALL OF THE PARAMETERS POSSIBLE TO FILL IN THE WORKSHEET. AI WILL USE THIS DATA TO CREATE OUTREACH SHOWING THAT WE'VE DONE RESEARCH.




# Geocode the town name to get its latitude and longitude
def geocode_town(town_name, api_key):
    geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': town_name,
        'key': api_key
    }
    response = requests.get(geocode_url, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result['results']:
            location = result['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print("No results found for town:", town_name)
            return None, None
    else:
        print("Geocoding failed with status:", response.status_code)
        return None, None

# Fetch nearby places from Google Places API
def get_nearby_places(api_key, location, radius, pagetoken=None):
    endpoint = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {
        'location': location,
        'radius': radius,
        'key': api_key
    }

    if pagetoken:
        params['pagetoken'] = pagetoken

    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Extract place data including website if available - if there is no website, don't include

def extract_place_data(api_key, results):
    data = []
    for place in results['results']:
        place_id = place.get('place_id')
        details = get_place_details(api_key, place_id)
        website = details.get('result', {}).get('website', None) if details else None  # Set website to None if not available
        
        # Skip businesses that don't have a website
        if website is None:
            continue  # Skip to the next place if there's no website
        
        # Get the types of the place
        types = ', '.join(place.get('types', []))  # Join the list of types as a string

        data.append({
            'Name': place.get('name'),
            'Address': place.get('vicinity'),
            'Rating': place.get('rating'),
            'Business Status': place.get('business_status'),
            'Place ID': place_id,
            'Website': website,  # Add website to the data
            'Type': types  # Add types to the data
        })
    return data


# Save the data to a CSV file
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Get all places in the area and handle pagination
def get_all_places(api_key, location, radius):
    all_places = []
    pagetoken = None
    while True:
        results = get_nearby_places(api_key, location, radius, pagetoken)
        if results and 'results' in results:
            place_data = extract_place_data(api_key, results)
            all_places.extend(place_data)
            
            # Check if there's a next page of results
            pagetoken = results.get('next_page_token')
            if pagetoken:
                print("Fetching next page of results...")
                time.sleep(2)  # Wait 2 seconds for the next page token to activate
            else:
                break
        else:
            break
    return all_places

# Search multiple areas by dividing the town into grids
def search_multiple_areas(api_key, base_lat, base_lng, radius, grid_size=4):
    all_places = []
    
    # Step size for dividing the area
    lat_step = 0.02  # Adjust based on the size of the town
    lng_step = 0.02  # Adjust based on the size of the town
    
    for i in range(grid_size):
        for j in range(grid_size):
            lat = base_lat + (i - grid_size//2) * lat_step
            lng = base_lng + (j - grid_size//2) * lng_step
            location = f"{lat},{lng}"
            print(f"Searching in area centered at {location}")
            places = get_all_places(api_key, location, radius)
            all_places.extend(places)
    
    return all_places

def main():
    api_key = google_place_api_key
    town_name = input("Enter the town name: ")

    # Get latitude and longitude from town name
    base_lat, base_lng = geocode_town(town_name, api_key)
    if base_lat is None or base_lng is None:
        print("Failed to get coordinates for the town.")
        return

    radius = 2000  # Set the radius to 2km for each search area
    grid_size = 5  # Divide the town into a 5x5 grid

    print(f"Searching for businesses in {town_name}...")
    
    # Search all areas in the town
    all_places = search_multiple_areas(api_key, base_lat, base_lng, radius, grid_size)

    if all_places:
        # Save to CSV
        save_to_csv(all_places, f'{town_name}_businesses.csv')
    else:
        print("No businesses found or an error occurred.")

if __name__ == '__main__':
    main()


Enter the town name:  los alamos, ca


Searching for businesses in los alamos, ca...
Searching in area centered at 34.7044264,-120.31821140000001
Searching in area centered at 34.7044264,-120.2982114
Searching in area centered at 34.7044264,-120.2782114
Searching in area centered at 34.7044264,-120.25821140000001
Searching in area centered at 34.7044264,-120.2382114
Searching in area centered at 34.7244264,-120.31821140000001
Searching in area centered at 34.7244264,-120.2982114
Searching in area centered at 34.7244264,-120.2782114
Searching in area centered at 34.7244264,-120.25821140000001
Searching in area centered at 34.7244264,-120.2382114
Searching in area centered at 34.7444264,-120.31821140000001
Searching in area centered at 34.7444264,-120.2982114
Fetching next page of results...
Fetching next page of results...
Searching in area centered at 34.7444264,-120.2782114
Fetching next page of results...
Fetching next page of results...
Searching in area centered at 34.7444264,-120.25821140000001
Fetching next page of re

In [ ]:
# From Zoominfo or whatever equivalent, Get company info, see how many people and add to database. Python Scrape, store as variable. use AI with vectorDB about PCIO to go through scrape and see if they're worthwhile. 

In [ ]:
#apollio.io, lusha, lead411

In [ ]:
#VECTOR DATABASE FOR PYTHON SCRAPE?